In [1]:
import sys

sys.executable

'/Users/marcelc/Documents/10_sem/pisr/pro/pisr_venv/bin/python'

In [338]:
import numpy as np
import pandas as pd
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from surprise import Dataset, SVD, Reader
from surprise.model_selection import cross_validate

pd.set_option('display.max_columns', None)

In [383]:
credits = pd.read_csv('../data/credits.csv')  # 45_476
keywords = pd.read_csv('../data/keywords.csv')  # 46_419
links_small = pd.read_csv('../data/links_small.csv')  # 9_125
# links = pd.read_csv('../data/links.csv')  # 45_843
movies = pd.read_csv('../data/movies_metadata.csv')  # 45_466
ratings_small = pd.read_csv('../data/ratings_small.csv')  # 100_004
# ratings = pd.read_csv('../data/ratings.csv')  # 26_024_289

### Credits dataframe preparation

In [384]:
credits['cast'] = credits['cast']\
    .apply(literal_eval)\
    .apply(lambda x: [i['name'].replace(' ', '_').lower() for i in x[:10]])

credits['crew'] = credits['crew']\
    .apply(literal_eval)\
    .apply(lambda x: [i['name'].replace(' ', '_').lower() for i in x[:10]])

In [385]:
print(credits.info())
credits.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB
None


,cast,crew,id
0,"[tom_hanks, tim_allen, don_rickles, jim_varney...","[john_lasseter, joss_whedon, andrew_stanton, j...",862
1,"[robin_williams, jonathan_hyde, kirsten_dunst,...","[larry_j._franco, jonathan_hensleigh, james_ho...",8844
2,"[walter_matthau, jack_lemmon, ann-margret, sop...","[howard_deutch, mark_steven_johnson, mark_stev...",15602
3,"[whitney_houston, angela_bassett, loretta_devi...","[forest_whitaker, ronald_bass, ronald_bass, ez...",31357
4,"[steve_martin, diane_keaton, martin_short, kim...","[alan_silvestri, elliot_davis, nancy_meyers, n...",11862


### Keywords dataframe preparation

In [386]:
keywords['keywords'] = keywords['keywords']\
    .apply(literal_eval)\
    .apply(lambda x: [i['name'].replace(' ', '_').lower() for i in x[:10]])

In [387]:
print(keywords.info())
keywords.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB
None


,id,keywords
0,862,"[jealousy, toy, boy, friendship, friends, riva..."
1,8844,"[board_game, disappearance, based_on_children'..."
2,15602,"[fishing, best_friend, duringcreditsstinger, o..."
3,31357,"[based_on_novel, interracial_relationship, sin..."
4,11862,"[baby, midlife_crisis, confidence, aging, daug..."


### Links dataframe preparation

In [388]:
links_small.columns = ['movie_id', 'imbd_id', 'tmdb_id']
links_small = links_small.dropna()

links_small['tmdb_id'] = links_small['tmdb_id'].astype('int')
links_small['movie_id'] = links_small['movie_id'].astype('int')

In [389]:
print(links_small.info())
links_small.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9112 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   movie_id  9112 non-null   int64
 1   imbd_id   9112 non-null   int64
 2   tmdb_id   9112 non-null   int64
dtypes: int64(3)
memory usage: 284.8 KB
None


,movie_id,imbd_id,tmdb_id
0,1,114709,862
1,2,113497,8844
2,3,113228,15602
3,4,114885,31357
4,5,113041,11862


### Movies dataframe preparation

In [390]:
to_drop = list(movies[
    movies['id'].isnull()
].index)

movies = movies.drop(to_drop)
movies = movies.drop([19730, 29503, 35587])

movies['id'] = movies['id'].astype('int')

movies = movies.rename(columns={'id': 'tmdb_id'})

movies['genres'] = movies['genres']\
    .apply(literal_eval)\
    .apply(lambda x: [i['name'] for i in x])

movies['year'] = movies['release_date']\
    .apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

movies['tagline'] = movies['tagline'].fillna('')
movies['overview'] = movies['overview'].fillna('')

movies['description'] = movies['overview'] + movies['tagline']
movies['description'] = movies['description'].fillna('')

# # collection column
# metadata['belongs_to_collection'].fillna('{\'name\': \'\'}', inplace=True)
# metadata['collection'] = metadata.progress_apply(lambda row: literal_eval(row['belongs_to_collection'])['name'], axis=1)

In [391]:
print(movies.info())
movies.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45463 entries, 0 to 45465
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45463 non-null  object 
 1   belongs_to_collection  4491 non-null   object 
 2   budget                 45463 non-null  object 
 3   genres                 45463 non-null  object 
 4   homepage               7779 non-null   object 
 5   tmdb_id                45463 non-null  int64  
 6   imdb_id                45446 non-null  object 
 7   original_language      45452 non-null  object 
 8   original_title         45463 non-null  object 
 9   overview               45463 non-null  object 
 10  popularity             45460 non-null  object 
 11  poster_path            45077 non-null  object 
 12  production_companies   45460 non-null  object 
 13  production_countries   45460 non-null  object 
 14  release_date           45376 non-null  object 
 15  re

,adult,belongs_to_collection,budget,genres,homepage,tmdb_id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,description
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,,Toy Story,False,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ..."
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,A family wedding reignites the ancient feud be...
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995,"Cheated on, mistreated and stepped on, the wom..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995,Just when George Banks has recovered from his ...


In [392]:
movies.head().transpose()

,0,1,2,3,4
adult,False,False,False,False,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', ...",NaN,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",NaN,"{'id': 96871, 'name': 'Father of the Bride Col..."
budget,30000000,65000000,0,16000000,0
genres,"[Animation, Comedy, Family]","[Adventure, Fantasy, Family]","[Romance, Comedy]","[Comedy, Drama, Romance]",[Comedy]
homepage,http://toystory.disney.com/toy-story,NaN,NaN,NaN,NaN
tmdb_id,862,8844,15602,31357,11862
imdb_id,tt0114709,tt0113497,tt0113228,tt0114885,tt0113041
original_language,en,en,en,en,en
original_title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II
overview,"Led by Woody, Andy's toys live happily in his ...",When siblings Judy and Peter discover an encha...,A family wedding reignites the ancient feud be...,"Cheated on, mistreated and stepped on, the wom...",Just when George Banks has recovered from his ...


### Ratings dataframe preparation

In [393]:
ratings_small.columns = ['user_id', 'movie_id', 'rating', 'timestamp']

In [394]:
print(ratings_small.info())
ratings_small.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user_id    100004 non-null  int64  
 1   movie_id   100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB
None


,user_id,movie_id,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


### Simple recomendation

IMDB's weighted rating formula:

$WR = (\frac{v}{v + m} . R) + (\frac{m}{v + m} . C)$
```
where,
    v is the number of votes for the movie
    m is the minimum votes required to be listed in the chart
    R is the average rating of the movie
    C is the mean vote across the whole report
```

In [395]:
# v
vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')

# R
vote_averages = movies[movies['vote_average'].notnull()]['vote_average']

# C
C = vote_averages.mean()

# m
m = vote_counts.quantile(0.9)

# print(f'v: {vote_counts}')
# print(f'R: {vote_averages}')
print(f'C: {C}')
print(f'm: {m}')

C: 5.618207215134184
m: 160.0


In [396]:
to_recommend = movies[
    (movies['vote_count'] >= m) & 
    (movies['vote_count'].notnull()) & 
    (movies['vote_average'].notnull())
][['title', 'year', 'original_language','vote_count', 
   'vote_average', 'popularity', 'genres']]

to_recommend.rename(columns={'original_language': 'language'}, inplace=True)

to_recommend['vote_count'] = to_recommend['vote_count'].astype('int')
to_recommend.shape

(4555, 7)

In [397]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    wr = (v/(v+m) * R) + (m/(m+v) * C)
    return wr


to_recommend['weighted_rating'] = to_recommend.apply(weighted_rating, axis=1)

to_recommend = to_recommend.sort_values('weighted_rating', ascending=False)

In [398]:
to_recommend.head(20)

,title,year,language,vote_count,vote_average,popularity,genres,weighted_rating
314,The Shawshank Redemption,1994,en,8358,8.5,51.645403,"[Drama, Crime]",8.445869
834,The Godfather,1972,en,6024,8.5,41.109264,"[Drama, Crime]",8.425439
10309,Dilwale Dulhania Le Jayenge,1995,hi,661,9.1,34.457024,"[Comedy, Drama, Romance]",8.421453
12481,The Dark Knight,2008,en,12269,8.3,123.167259,"[Drama, Action, Crime, Thriller]",8.265477
2843,Fight Club,1999,en,9678,8.3,63.869599,[Drama],8.256385
292,Pulp Fiction,1994,en,8670,8.3,140.950236,"[Thriller, Crime]",8.251406
522,Schindler's List,1993,en,4436,8.3,41.725123,"[Drama, History, War]",8.206639
23673,Whiplash,2014,en,4376,8.3,64.29999,[Drama],8.205404
5481,Spirited Away,2001,ja,3968,8.3,41.048867,"[Fantasy, Adventure, Animation, Family]",8.196055
2211,Life Is Beautiful,1997,it,3643,8.3,39.39497,"[Comedy, Drama]",8.187171


### Collaborative Filtering

In [399]:
# reader = Reader(rating_scale=(0.5, 5)) 
reader = Reader()

ratings_df = Dataset.load_from_df(ratings_small[['user_id', 'movie_id', 'rating']], reader)

svd = SVD()

# Run 5-fold cross-validation and print results
cross_validate(svd, ratings_df, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8890  0.9027  0.9017  0.8996  0.8926  0.8971  0.0054  
MAE (testset)     0.6839  0.6932  0.6918  0.6926  0.6881  0.6899  0.0035  
Fit time          5.06    5.06    5.05    5.05    5.09    5.06    0.01    
Test time         0.13    0.13    0.13    0.13    0.14    0.13    0.00    


{'test_rmse': array([0.88900472, 0.90269948, 0.90174487, 0.89956323, 0.89256907]),
 'test_mae': array([0.68388952, 0.69320802, 0.69179952, 0.69261517, 0.68812736]),
 'fit_time': (5.063357830047607,
  5.064008951187134,
  5.0544421672821045,
  5.052011966705322,
  5.090696811676025),
 'test_time': (0.13465380668640137,
  0.13336610794067383,
  0.1346750259399414,
  0.1337568759918213,
  0.13822484016418457)}

In [400]:
trainset = ratings_df.build_full_trainset()

svd.fit(trainset)

### Content-based

In [401]:
movies_in_links = movies[
    movies['tmdb_id'].isin(links_small['tmdb_id'])
]

movies_in_links.shape

(9099, 26)

In [402]:
movies_in_links.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9099 entries, 0 to 45265
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  9099 non-null   object 
 1   belongs_to_collection  1674 non-null   object 
 2   budget                 9099 non-null   object 
 3   genres                 9099 non-null   object 
 4   homepage               1974 non-null   object 
 5   tmdb_id                9099 non-null   int64  
 6   imdb_id                9099 non-null   object 
 7   original_language      9099 non-null   object 
 8   original_title         9099 non-null   object 
 9   overview               9099 non-null   object 
 10  popularity             9099 non-null   object 
 11  poster_path            9096 non-null   object 
 12  production_companies   9099 non-null   object 
 13  production_countries   9099 non-null   object 
 14  release_date           9099 non-null   object 
 15  rev

In [403]:
tf = TfidfVectorizer(
    analyzer='word', 
    ngram_range=(1, 2), 
    min_df=0, 
    stop_words='english'
)

tfidf_matrix = tf.fit_transform(movies_in_links['description'])

In [404]:
tfidf_matrix.shape

(9099, 268124)

In [405]:
# http://scikit-learn.org/stable/modules/metrics.html#linear-kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

cosine_sim.shape

(9099, 9099)

In [406]:
movies_in_links = movies_in_links.reset_index()
titles = movies_in_links['title']
indices = pd.Series(movies_in_links.index, index=movies_in_links['title'])

indices.head()

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64

### Hybrid recommendation system

In [407]:
id_map = links_small[['movie_id', 'tmdb_id']]
id_map = id_map.merge(movies_in_links[['title', 'tmdb_id']], on='tmdb_id').set_index('title')

In [409]:
indices_map = id_map.set_index('tmdb_id')

In [415]:
def hybrid(userId, title):
    idx = indices[title]
    tmdb_id = id_map.loc[title]['tmdb_id']
    movie_id = id_map.loc[title]['movie_id']

    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]

    movie_indices = [i[0] for i in sim_scores]
    movies = movies_in_links.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'release_date', 'tmdb_id']]
    movies['est'] = movies['tmdb_id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movie_id']).est)
    movies = movies.sort_values('est', ascending=False)

    return movies.head(10)

In [430]:
ratings[
    ratings['user_id'] == 3
]

,user_id,movie_id,rating,timestamp
49,3,480,3.0,1048076925
50,3,500,2.0,1048076945
51,3,527,4.0,1048076900
52,3,858,4.0,1048076945
53,3,1092,3.0,1048076995
54,3,1097,3.0,1048076961
55,3,1270,3.0,1048076976
56,3,1968,3.0,1048077048
57,3,3101,3.0,1048076830
58,3,4474,3.0,1048077030


In [431]:
hybrid(3, 'Toy Story')

,title,vote_count,vote_average,release_date,tmdb_id,est
865,Sleeper,206.0,7.0,1973-12-17,11561,3.847359
889,Rebel Without a Cause,351.0,7.6,1955-10-27,221,3.822656
1938,Stepmom,286.0,6.9,1998-12-25,9441,3.766240
7535,Toy Story 3,4710.0,7.6,2010-06-16,10193,3.748644
6193,The 40 Year Old Virgin,2020.0,6.2,2005-08-11,6957,3.633117
4988,Rivers and Tides,13.0,7.3,2001-02-13,30140,3.605525
2502,Toy Story 2,3914.0,7.3,1999-10-30,863,3.569366
994,Manhattan,600.0,7.8,1979-04-25,696,3.563979
6627,Factory Girl,83.0,6.2,2006-12-29,12271,3.490530
1599,Condorman,37.0,5.6,1981-07-02,19379,3.484505


In [420]:
hybrid(1, 'Toy Story')

,title,vote_count,vote_average,release_date,tmdb_id,est
865,Sleeper,206.0,7.0,1973-12-17,11561,3.168956
7535,Toy Story 3,4710.0,7.6,2010-06-16,10193,3.116075
994,Manhattan,600.0,7.8,1979-04-25,696,3.070713
2502,Toy Story 2,3914.0,7.3,1999-10-30,863,3.050296
8330,Woody Allen: A Documentary,56.0,7.5,2011-11-20,81167,3.032542
889,Rebel Without a Cause,351.0,7.6,1955-10-27,221,3.005200
4988,Rivers and Tides,13.0,7.3,2001-02-13,30140,2.897252
7723,Burke & Hare,151.0,6.1,2010-10-29,50204,2.891270
1938,Stepmom,286.0,6.9,1998-12-25,9441,2.872729
6193,The 40 Year Old Virgin,2020.0,6.2,2005-08-11,6957,2.842066


In [432]:
hybrid(3, 'The Godfather')

,title,vote_count,vote_average,release_date,tmdb_id,est
2412,American Movie,57.0,7.7,1999-11-05,14242,4.068001
973,The Godfather: Part II,3418.0,8.3,1974-12-20,240,3.879644
2192,The Color Purple,345.0,7.7,1985-12-18,873,3.855531
8387,The Family,1052.0,6.1,2013-09-13,112205,3.712366
7591,Machete,1171.0,6.3,2010-09-01,23631,3.711688
29,Shanghai Triad,17.0,6.5,1995-04-30,37557,3.680377
3609,Harlem Nights,85.0,5.7,1989-11-17,9085,3.550247
7760,Henry's Crime,110.0,5.8,2010-01-14,53172,3.516939
3560,Moon Over Parador,17.0,6.0,1988-09-09,34014,3.467828
5667,Fury,38.0,7.5,1936-05-29,14615,3.464615


In [422]:
hybrid(1, 'The Godfather')

,title,vote_count,vote_average,release_date,tmdb_id,est
973,The Godfather: Part II,3418.0,8.3,1974-12-20,240,3.314245
2412,American Movie,57.0,7.7,1999-11-05,14242,3.270059
2192,The Color Purple,345.0,7.7,1985-12-18,873,3.195360
5593,Eulogy,34.0,6.4,2004-10-15,16358,2.967622
29,Shanghai Triad,17.0,6.5,1995-04-30,37557,2.955071
8387,The Family,1052.0,6.1,2013-09-13,112205,2.928162
7591,Machete,1171.0,6.3,2010-09-01,23631,2.859331
5406,The Kid Brother,13.0,7.5,1927-01-17,16661,2.838179
5667,Fury,38.0,7.5,1936-05-29,14615,2.806924
4221,8 Women,197.0,6.9,2002-01-08,1958,2.766160


In [421]:
to_recommend.head(20)

,title,year,language,vote_count,vote_average,popularity,genres,weighted_rating
314,The Shawshank Redemption,1994,en,8358,8.5,51.645403,"[Drama, Crime]",8.445869
834,The Godfather,1972,en,6024,8.5,41.109264,"[Drama, Crime]",8.425439
10309,Dilwale Dulhania Le Jayenge,1995,hi,661,9.1,34.457024,"[Comedy, Drama, Romance]",8.421453
12481,The Dark Knight,2008,en,12269,8.3,123.167259,"[Drama, Action, Crime, Thriller]",8.265477
2843,Fight Club,1999,en,9678,8.3,63.869599,[Drama],8.256385
292,Pulp Fiction,1994,en,8670,8.3,140.950236,"[Thriller, Crime]",8.251406
522,Schindler's List,1993,en,4436,8.3,41.725123,"[Drama, History, War]",8.206639
23673,Whiplash,2014,en,4376,8.3,64.29999,[Drama],8.205404
5481,Spirited Away,2001,ja,3968,8.3,41.048867,"[Fantasy, Adventure, Animation, Family]",8.196055
2211,Life Is Beautiful,1997,it,3643,8.3,39.39497,"[Comedy, Drama]",8.187171
